# Model Inference

## Prepare Library, Parameters, and Dependencies

In [20]:
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image

In [21]:
# UNCOMMENT THIS IF YOU USE GOOGLE COLAB

# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
# Check TensorFlow Version and GPU Availability
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.14.0
Num GPUs Available:  0


In [ ]:
# Set Parameters
SEGMENTATION_MODEL_PATH = "models/segmentation_model_1/tflite/model.tflite"
CLASSIFICATION_MODEL_PATH = "models/classification_model_1/tflite/model.tflite"
TEST_PREDICT_DIR = 'datasets/test_predict'

## Load TFLite Model and Predict Image Path

In [ ]:
# Load TF Lite model
interpreter_segmentation = tf.lite.Interpreter(SEGMENTATION_MODEL_PATH)
interpreter_classification = tf.lite.Interpreter(CLASSIFICATION_MODEL_PATH)

In [ ]:
# Load Predict Image Path
image_path = []
for root, dirs, files in os.walk(TEST_PREDICT_DIR):
    for file in files:
        path_og = os.path.join(root,file)
        image_path.append(path_og)

## Run Inference

In [ ]:
# Allocate Tensors for Input and Output of Model
interpreter_segmentation.allocate_tensors()
interpreter_classification.allocate_tensors()

In [ ]:
# Get Height and Width of Input Image
_, height, width, _ = interpreter_segmentation.get_input_details()[0]['shape']

In [ ]:
# Run Inference for Each Image
for path in image_path:
    # Load Image from Path and Resize
    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (width, height)
    
    # Preprocess Image
    input_image = np.expand_dims(np.array(image), axis=0)
    
    # Set Input and Invoke Model
    interpreter_segmentation.set_tensor(interpreter_segmentation.get_input_details()[0]['index'], input_image)
    interpreter_segmentation.invoke()
    
    # Get Output
    output_segmentation = interpreter_segmentation.get_tensor(interpreter_segmentation.get_output_details()[0]['index'])
    
    # Get Mask
    mask = np.argmax(output_segmentation[0], axis=-1)
    mask = np.expand_dims(mask, axis=-1)
    
    # Get Image with Mask
    image_with_mask = np.array(image)
    image_with_mask[mask == 0] = 0
    
    # Preprocess Image with Mask
    input_image_with_mask = np.expand_dims(np.array(image_with_mask), axis=0)
    
    # Set Input and Invoke Model
    interpreter_classification.set_tensor(interpreter_classification.get_input_details()[0]['index'], input_image_with_mask)
    interpreter_classification.invoke()
    
    # Get Output
    output_classification = interpreter_classification.get_tensor(interpreter_classification.get_output_details()[0]['index'])
    
    # Get Prediction
    prediction = np.argmax(output_classification[0], axis=-1)
    
    # Save Prediction
    if prediction == 0:
        prediction = ''
    elif prediction == 1:
        prediction = 'normal'
    elif prediction == 2:
        prediction = 'viral'
        
    # Save Image with Mask
    image_with_mask = Image.fromarray(image_with_mask)
    image_with_mask.save(path.replace('test_predict', 'test_predict_with_mask'))
    
    # Save Prediction
    with open(path.replace('test_predict', 'test_predict_with_mask').replace('.jpg', '.txt'), 'w') as f:
        f.write(prediction)